<a href="https://colab.research.google.com/github/galenzo17/AI-personal-test/blob/main/local_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U llama-index-core

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.0 MB/s eta 0:00:00


In [2]:
!pip install llama-index-embeddings-huggingface llama-agents

INFO: pip is looking at multiple versions of llama-agents to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of llama-agents to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 562.2/562.2 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 4.3 M

In [3]:
import logging

# Create a logger instance
logger = logging.getLogger(__name__)

# Set the log level to INFO for informational messages
logger.setLevel(logging.INFO)

In [4]:
import nest_asyncio

nest_asyncio.apply()

#import os

#os.environ["OPENAI_API_KEY"] = "sk-proj-..."

In [5]:
from llama_agents import (
    AgentService,
    AgentOrchestrator,
    ControlPlaneServer,
    LocalLauncher,
    SimpleMessageQueue,
)

from llama_index.core.agent import ReActAgent
from llama_index.core.tools import FunctionTool
from llama_index.llms.ollama import Ollama
from llama_index.core import Settings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.agent import FunctionCallingAgentWorker


# Calculator tool
def calculator(operation: str, a: float, b: float) -> str:
    """
    Perform basic arithmetic operations.

    Args:
    operation (str): One of 'add', 'subtract', 'multiply', or 'divide'.
    a (float): First number.
    b (float): Second number.

    Returns:
    str: Result of the operation as a string.
    """
    try:
        if operation == "add":
            result = a + b
        elif operation == "subtract":
            result = a - b
        elif operation == "multiply":
            result = a * b
        elif operation == "divide":
            if b == 0:
                return "Error: Cannot divide by zero."
            result = a / b
        else:
            return f"Error: Invalid operation '{operation}'. Choose 'add', 'subtract', 'multiply', or 'divide'."

        return f"The result of {a} {operation} {b} is {result}"
    except Exception as e:
        return f"Error in calculation: {str(e)}"

# New text analysis tool
def text_analyzer(text: str) -> str:
    """
    Perform basic text analysis.

    Args:
    text (str): The text to analyze.

    Returns:
    str: Analysis results as a string.
    """
    try:
        word_count = len(text.split())
        char_count = len(text)
        sentence_count = text.count('.') + text.count('!') + text.count('?')

        analysis = f"Text Analysis Results:\n"
        analysis += f"- Word count: {word_count}\n"
        analysis += f"- Character count: {char_count}\n"
        analysis += f"- Approximate sentence count: {sentence_count}\n"

        return analysis
    except Exception as e:
        return f"Error in text analysis: {str(e)}"

calculator_tool = FunctionTool.from_defaults(fn=calculator)
text_tool = FunctionTool.from_defaults(fn=text_analyzer)


llm = Ollama(model="arcee-ai/arcee-agent", request_timeout=120.0)

Settings.llm = llm



agent1 = ReActAgent.from_tools(tools=[calculator_tool], llm=llm, verbose=True)
agent2 = ReActAgent.from_tools([text_tool], llm=Ollama(model="mistral", request_timeout=120.0), verbose=True)

# worker2 = FunctionCallingAgentWorker.from_tools([text_tool], llm=llm)
# agent2 = worker2.as_agent()


Settings.embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-small-en-v1.5"
)

# create our multi-agent framework components
message_queue = SimpleMessageQueue(port=8000)
control_plane = ControlPlaneServer(
    message_queue=message_queue,
    orchestrator=AgentOrchestrator(llm=llm),
    port=8001,

)
agent_server_1 = AgentService(
    agent=agent1,
    message_queue=message_queue,
    description="Useful for performing basic arithmetic operations like calculations.",
    service_name="calculator_agent",
    port=8002,
)
agent_server_2 = AgentService(
    agent=agent2,
    message_queue=message_queue,
    description="Useful for performing NLP, Text Analysis and Text Processing.",
    service_name="nlp_agent",
    port=8003,
)

# launch it
launcher = LocalLauncher([agent_server_1, agent_server_2], control_plane, message_queue)

try:
    result = launcher.launch_single("can you divide 100 by 20?")


    print(f"Result: {result}")
except Exception as e:
    print(f"An error occurred: {str(e)}")
    print("Please check the agent logs for more details.")

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /usr/local/lib/python3.10/dist-
[nltk_data]     packages/llama_index/core/_static/nltk_cache...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


ModuleNotFoundError: No module named 'llama_index.llms'